In [ ]:
%pip install gpt4all > /dev/null

In [ ]:
pip install langchain

In [ ]:
!pip install pypdf


In [ ]:
!pip install chromadb

In [ ]:
!pip install openai

In [ ]:
pip install tiktoken

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 21.0 MB/s eta 0:00:00


In [ ]:
from langchain import PromptTemplate, LLMChain
from langchain.llms import GPT4All
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

In [ ]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [ ]:
local_path = (
    "./models/ggml-gpt4all-l13b-snoozy.bin"  # replace with your desired local file path
)

import requests
from pathlib import Path
from tqdm import tqdm

Path(local_path).parent.mkdir(parents=True, exist_ok=True)
# Example model. Check https://github.com/nomic-ai/gpt4all for the latest models.
url = 'http://gpt4all.io/models/ggml-gpt4all-l13b-snoozy.bin'

# # send a GET request to the URL to download the file. Stream since it's large
response = requests.get(url, stream=True)

# # open the file in binary mode and write the contents of the response to it in chunks
# # This is a large file, so be prepared to wait.
with open(local_path, 'wb') as f:
     for chunk in tqdm(response.iter_content(chunk_size=8192)):
         if chunk:
             f.write(chunk)

993259it [03:30, 4714.80it/s]


In [ ]:
# Callbacks support token-wise streaming
callbacks = [StreamingStdOutCallbackHandler()]

# Verbose is required to pass to the callback manager
llm = GPT4All(model=local_path, callbacks=callbacks, verbose=True)

# If you want to use a custom model add the backend parameter
# Check https://docs.gpt4all.io/gpt4all_python.html for supported backends
llm = GPT4All(model=local_path, backend="gptj", callbacks=callbacks, verbose=True)

Found model file at  ./models/ggml-gpt4all-l13b-snoozy.bin
Found model file at  ./models/ggml-gpt4all-l13b-snoozy.bin


In [ ]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [ ]:
question = "What NFL team won the Super Bowl in the year Justin Bieber was born?"

llm_chain.run(question)

 First, we need to know when Justin Bieber was born (March 1st, 1994). Then, we can look up which teams played in each of the Super Bowls that took place on or before March 1st, 1994:
- The New York Giants won Super Bowl XXI.
- The Pittsburgh Steelers won Super Bowl XIII.
- The Dallas Cowboys won Super Bowl XII.
- The Oakland Raiders won Super Bowl XI.
- The Green Bay Packers won Super Bowl X.
- The Baltimore Colts (now the Indianapolis Colts) won Super Bowl V.
- The Kansas City Chiefs won Super Bowl IV.
- The Cleveland Browns won Super Bowl II.

' First, we need to know when Justin Bieber was born (March 1st, 1994). Then, we can look up which teams played in each of the Super Bowls that took place on or before March 1st, 1994:\n- The New York Giants won Super Bowl XXI.\n- The Pittsburgh Steelers won Super Bowl XIII.\n- The Dallas Cowboys won Super Bowl XII.\n- The Oakland Raiders won Super Bowl XI.\n- The Green Bay Packers won Super Bowl X.\n- The Baltimore Colts (now the Indianapolis Colts) won Super Bowl V.\n- The Kansas City Chiefs won Super Bowl IV.\n- The Cleveland Browns won Super Bowl II.'

In [ ]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.document_loaders import PyPDFLoader
from langchain.embeddings import LlamaCppEmbeddings

In [ ]:
import os
import getpass

os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key:')

OpenAI API Key:··········


In [ ]:
loader = PyPDFLoader("/content/data_Anlasma_Metni.pdf")
documents = loader.load_and_split()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(documents)

embeddings = OpenAIEmbeddings()
docsearch = Chroma.from_documents(texts, embeddings)

qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=docsearch.as_retriever())

In [ ]:
query = "What did the president say about Ketanji Brown Jackson"
qa.run(query)

ERROR: The prompt size exceeds the context window size and cannot be processed.

'ERROR: The prompt size exceeds the context window size and cannot be processed.'

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.chains.qa_with_sources import load_qa_with_sources_chain

In [ ]:
template = """Compose a final answer by quoting the following quotes from a long document and a question ("RESOURCES").
If you don't know the answer, say you don't. Don't try to fabricate an answer.
ALWAYS include a "RESOURCES" section in your answer with the page number, pdf name, and line spacing with the answer.

QUESTION: {question}
=========
{summaries}
=========
FINAL :"""
PROMPT = PromptTemplate(template=template, input_variables=["summaries", "question"])

chain = load_qa_with_sources_chain(llm, chain_type="stuff", prompt=PROMPT)
query = "taraflar kimlerdir?"
chain({"input_documents": documents, "question": query }, return_only_outputs=True)

ERROR: The prompt size exceeds the context window size and cannot be processed.

{'output_text': 'ERROR: The prompt size exceeds the context window size and cannot be processed.'}